# **4A**

In [2]:
# 4a
import time
import multiprocessing as mp
import numpy as np

def multiplicar_matrices(matriz):
    A = matriz
    # Generar matriz B con dimensiones compatibles con A
    B = np.random.rand(*A.shape)  # B tendrá las mismas dimensiones que A
    return np.dot(A, B)

def ejecutar_con_pool(num_procesos, tamaño_matriz):
    matriz_ejemplo = np.random.rand(*tamaño_matriz)
    with mp.Pool(processes=num_procesos) as pool:
        resultados = pool.map(multiplicar_matrices, [matriz_ejemplo] * num_procesos)
    return resultados

# Tamaño de la matriz y número de procesos para pruebas
tamaño_matriz = (100, 100)  # Ejemplo de tamaño de matriz más pequeño
num_procesos = mp.cpu_count() * 7  # Multiplicar por 7 veces el número de procesadores disponibles

# Ejecutar y medir el tiempo de ejecución
start_time = time.time()
resultados = ejecutar_con_pool(num_procesos, tamaño_matriz)
end_time = time.time()

tiempo_ejecucion = end_time - start_time

print(f"Tiempo de ejecución para Pool con {num_procesos} procesos: {tiempo_ejecucion} segundos")

Tiempo de ejecución para Pool con 14 procesos: 0.46199774742126465 segundos


# **4B**

In [3]:
# 4b
def multiplicar_matrices_chunk(chunk):
    A, B = chunk
    return np.dot(A, B)

def multiplicar_matrices_chunks(num_chunks, tamaño_matriz):
    # Generar matriz A y matriz B con dimensiones compatibles
    A = np.random.rand(*tamaño_matriz)
    B = np.random.rand(*tamaño_matriz)

    # Dividir A y B en chunks según el número de chunks especificado
    chunks_A = np.array_split(A, num_chunks, axis=0)
    chunks_B = np.array_split(B, num_chunks, axis=1)

    # Empaquetar cada par de chunks A y B
    chunks = [(chunks_A[i], chunks_B[i]) for i in range(num_chunks)]

    # Utilizar multiprocessing para multiplicar las matrices en paralelo
    with mp.Pool(processes=num_chunks) as pool:
        resultados = pool.map(multiplicar_matrices_chunk, chunks)

    # Concatenar los resultados de cada chunk en una única matriz
    resultado_final = np.concatenate(resultados)

    return resultado_final

# Tamaño de la matriz y número de chunks para pruebas
tamaño_matriz = (2000, 2000)
num_chunks_list = [1, 2, 4, 8, 16]  # Variar el número de chunks

# Ejecutar y medir el tiempo de ejecución para cada configuración
for num_chunks in num_chunks_list:
    start_time = time.time()
    resultado = multiplicar_matrices_chunks(num_chunks, tamaño_matriz)
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time

    print(f"Tiempo de ejecución para {num_chunks} chunks: {tiempo_ejecucion:.4f} segundos")


Tiempo de ejecución para 1 chunks: 1.7509 segundos
Tiempo de ejecución para 2 chunks: 1.2333 segundos
Tiempo de ejecución para 4 chunks: 1.0390 segundos
Tiempo de ejecución para 8 chunks: 0.7903 segundos
Tiempo de ejecución para 16 chunks: 0.8836 segundos


# **4C**

Para encontrar la combinación idónea entre el número de procesos y la medida de los chunks que minimice el tiempo de ejecución en la multiplicación de matrices, podemos basarnos en los tiempos de ejecución obtenidos en las partes anteriores del ejercicio 4a y 4b. Aquí está el proceso seguido y las decisiones tomadas:

**Análisis de los tiempos de ejecución:**

1. **Tiempo de ejecución para Pool con 14 procesos:** 0.462 segundos
   - Este tiempo nos da una referencia de cuánto toma ejecutar el código con 14 procesos. Es un tiempo bastante bajo, lo cual indica una buena eficiencia en paralelismo.

2. **Tiempo de ejecución variando el número de chunks:**
   - 1 chunk: 1.7509 segundos
   - 2 chunks: 1.2333 segundos
   - 4 chunks: 1.0390 segundos
   - 8 chunks: 0.7903 segundos
   - 16 chunks: 0.8836 segundos

**Razonamiento y decisiones:**

- **Número de procesos:** Basado en el tiempo de ejecución del Pool con 14 procesos (0.462 segundos), podemos inferir que utilizar un número cercano a 14 procesos es eficiente. Sin embargo, el tiempo de ejecución no se reduce linealmente con el número de procesos debido a otros factores de sobrecarga y administración del sistema.

- **Número de chunks:** Observamos que a medida que aumentamos el número de chunks, el tiempo de ejecución disminuye hasta cierto punto. Luego, para un número mayor de chunks (más de 8 en este caso), el tiempo parece estabilizarse o incluso aumentar ligeramente. Esto puede deberse a la sobrecarga adicional de dividir y gestionar los chunks más pequeños.

**Combinación idónea:**

Para determinar la combinación idónea que minimice el tiempo de ejecución:

- **Procesos:** Podemos considerar utilizar alrededor de 14 procesos, como se observó en la ejecución con Pool.
- **Chunks:** Según los tiempos obtenidos, el tiempo más bajo se obtuvo con 8 chunks (0.7903 segundos). Aunque 16 chunks también tiene un tiempo similar (0.8836 segundos), puede ser beneficioso elegir un número ligeramente menor de chunks para reducir la complejidad de la gestión de los chunks.

**Decisión final:**

Una combinación razonable basada en los datos sería utilizar **14 procesos** con **8 chunks**. Esto equilibra el paralelismo proporcionado por los procesos con la eficiencia obtenida al dividir las matrices en chunks de tamaño moderado.

Implementar esta combinación en el código y medir el tiempo de ejecución confirmará si es la más óptima para tu caso específico. Si se requiere mayor precisión, también se pueden realizar pruebas adicionales variando ligeramente el número de procesos o chunks y comparando los tiempos resultantes.

# **4D**

Para calcular y analizar los parámetros T1, T∞, Tp, Sp y los recursos gastados para las ejecuciones del apartado 4, utilizaremos la información obtenida de los tiempos de ejecución para diferentes configuraciones de procesos y chunks. Aquí vamos a calcular y luego utilizar estos parámetros en nuestros razonamientos y decisiones:

### Definición de términos:

- **T1:** Tiempo de ejecución con un solo proceso.
- **T∞:** Tiempo de ejecución con infinitos procesos, es decir, el límite inferior teórico del tiempo de ejecución.
- **Tp:** Tiempo de ejecución con p procesos.
- **Sp:** Aceleración de la ejecución utilizando p procesos en comparación con un solo proceso (Sp = T1 / Tp).
- **Recursos gastados:** Refiere a la cantidad de recursos computacionales utilizados, que puede medirse en términos de memoria, CPU, tiempo de ejecución total, etc.

### Pasos para el cálculo y análisis:

1. **Calcular T1:** Utilizaremos el tiempo de ejecución con un solo proceso. Asumiremos que este es el tiempo mínimo teórico para la tarea (aunque en realidad podría ser más alto debido a la sobrecarga general del sistema).

2. **Calcular T∞:** Aunque no es posible ejecutar con infinitos procesos, T∞ se considera el límite inferior teórico del tiempo de ejecución, que normalmente se aproxima al tiempo que tomaría la operación sin considerar ningún tipo de sobrecarga adicional.

3. **Calcular Tp para cada configuración:** Utilizaremos los tiempos de ejecución medidos para 14 procesos y diferentes cantidades de chunks (1, 2, 4, 8, 16).

4. **Calcular Sp para cada configuración:** Sp se calcula como la relación entre T1 y Tp (Sp = T1 / Tp). Este nos indicará cuánto más rápido es el procesamiento paralelo en comparación con el procesamiento secuencial.

5. **Analizar recursos gastados:** Esto puede implicar revisar el uso de CPU, memoria, y cualquier otra métrica que sea relevante para determinar la eficiencia y la escalabilidad del sistema.

### Ejemplo de cálculo:

Tomaremos como ejemplo los tiempos de ejecución proporcionados anteriormente:

- T1 (tiempo con un solo proceso): 1.7509 segundos (tomando el tiempo con 1 chunk).
- T∞ (tiempo teórico con infinitos procesos): No se puede calcular directamente, pero se considera el límite inferior teórico.
- Tp para diferentes configuraciones (14 procesos y diferentes números de chunks):
  - Para 1 chunk: Tp = 1.7509 segundos
  - Para 2 chunks: Tp = 1.2333 segundos
  - Para 4 chunks: Tp = 1.0390 segundos
  - Para 8 chunks: Tp = 0.7903 segundos
  - Para 16 chunks: Tp = 0.8836 segundos

### Calcular Sp para cada configuración:

- Sp para 1 chunk: Sp = T1 / Tp = 1.7509 / 1.7509 = 1 (sin aceleración, pues es secuencial).
- Sp para 2 chunks: Sp = 1.7509 / 1.2333 ≈ 1.42 (aproximadamente 1.42 veces más rápido).
- Sp para 4 chunks: Sp = 1.7509 / 1.0390 ≈ 1.68 (aproximadamente 1.68 veces más rápido).
- Sp para 8 chunks: Sp = 1.7509 / 0.7903 ≈ 2.22 (aproximadamente 2.22 veces más rápido).
- Sp para 16 chunks: Sp = 1.7509 / 0.8836 ≈ 1.98 (aproximadamente 1.98 veces más rápido).

### Análisis de recursos gastados:

- Se debería considerar la carga de trabajo por proceso, la utilización de la memoria, y cualquier sobrecarga adicional del sistema al utilizar más procesos y chunks.

### Toma de decisiones:

- Basado en los resultados de Sp, se puede decidir qué configuración ofrece el mejor rendimiento en términos de velocidad de procesamiento.
- Considerar también los recursos gastados para determinar cuál es la configuración más eficiente desde el punto de vista computacional.

Utilizando estos parámetros, podemos hacer una elección más informada sobre la configuración óptima de procesos y chunks para minimizar el tiempo de ejecución y optimizar el uso de recursos en la multiplicación de matrices.